# ⚠️ FROZEN NOTEBOOK

This notebook is used **only** for offline learning of feature importance
from paired clean–dyslexic Sinhala sentences.

- ❌ Not used for runtime inference
- ❌ Not used for proxy reference generation
- ✅ Outputs learned feature weights
- ✅ Feeds rule-based inference system

Any changes require retraining and re-validation.


In [1]:
import sys
import os
print(sys.version)
print(os.getcwd())


3.12.3 (tags/v3.12.3:f6650f9, Apr  9 2024, 14:05:25) [MSC v.1938 64 bit (AMD64)]
d:\RP\sinhala-dyslexic-writing-pattern-classifier\writing_pattern_classifier\notebooks


In [3]:
import sys
import os

# This MUST point to the repo root
PROJECT_ROOT = os.path.abspath(os.path.join(os.getcwd(), "..", ".."))

if PROJECT_ROOT not in sys.path:
    sys.path.insert(0, PROJECT_ROOT)

print("Project root added:", PROJECT_ROOT)


Project root added: d:\RP\sinhala-dyslexic-writing-pattern-classifier


In [4]:
from writing_pattern_classifier.src.feature_extraction import extract_surface_features_offline


In [5]:
from datasets import load_dataset
import pandas as pd

ds = load_dataset("SPEAK-ASR/akura-sinhala-dyslexia-corrected")
df = ds["train"].to_pandas()

df.head()


,clean_sentence,dyslexic_sentence,error_type
0,ගානට සල්ලි,ගාණට සල්ලි,Phonetic Confusion
1,එයා බය වුණාද?,එයා බය උනාද,Spelling
2,පොත බලනවා.,පොත බලනව.,grammar
3,කඩෙන් පාන් ගත්තා,කඩෙන් පව් ගත්ත,phonetic_confusion
4,සමන් හවස ඇවිත් ගියා.,සමන් හවසා ඇවිත් ගියා.,Visual Scrambling


In [6]:
feature_rows = df.apply(
    lambda r: extract_surface_features_offline(
        r["clean_sentence"],
        r["dyslexic_sentence"]
    ),
    axis=1
)

feature_df = pd.concat(
    [df.reset_index(drop=True), feature_rows.apply(pd.Series)],
    axis=1
)

feature_df.head()


,clean_sentence,dyslexic_sentence,error_type,char_addition,char_omission,char_substitution,has_addition,has_omission,has_substitution,word_count_diff,has_spacing_issue,has_diacritic_loss
0,ගානට සල්ලි,ගාණට සල්ලි,Phonetic Confusion,0,0,1,False,False,True,0,False,False
1,එයා බය වුණාද?,එයා බය උනාද,Spelling,0,1,3,False,True,True,0,False,True
2,පොත බලනවා.,පොත බලනව.,grammar,0,1,0,False,True,False,0,False,True
3,කඩෙන් පාන් ගත්තා,කඩෙන් පව් ගත්ත,phonetic_confusion,0,1,2,False,True,True,0,False,True
4,සමන් හවස ඇවිත් ගියා.,සමන් හවසා ඇවිත් ගියා.,Visual Scrambling,1,0,0,True,False,False,0,False,False


In [7]:
LABEL_MAP = {
    "phonetic_confusion": "Phonetic Confusion",
    "Phonetic Confusion": "Phonetic Confusion",
    "phonetic": "Phonetic Confusion",

    "grammar": "Orthographic Instability",
    "Grammar": "Orthographic Instability",
    "Spelling": "Orthographic Instability",

    "visual_scrambling": "Word Boundary Confusion",
    "Visual Scrambling": "Word Boundary Confusion",
    "visual_reversal": "Word Boundary Confusion",
}

feature_df["label"] = feature_df["error_type"].map(LABEL_MAP)


In [8]:
feature_df = feature_df.dropna(subset=["label"]).reset_index(drop=True)

feature_df["label"].value_counts()


label
Orthographic Instability    7849
Phonetic Confusion          4632
Word Boundary Confusion     1709
Name: count, dtype: int64

In [9]:
FEATURE_COLS = [
    "char_addition",
    "char_omission",
    "char_substitution",
    "has_addition",
    "has_omission",
    "has_substitution",
    "word_count_diff",
    "has_spacing_issue",
    "has_diacritic_loss"
]

X = feature_df[FEATURE_COLS]
y = feature_df["label"]


In [10]:
X = X.fillna(0)


In [11]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X,
    y,
    test_size=0.2,
    random_state=42,
    stratify=y
)


In [12]:
from sklearn.linear_model import LogisticRegression

model = LogisticRegression(max_iter=1000)
model.fit(X_train, y_train)


,"penalty penalty: {'l1', 'l2', 'elasticnet', None}, default='l2'Specify the norm of the penalty:- `None`: no penalty is added;- `'l2'`: add a L2 penalty term and it is the default choice;- `'l1'`: add a L1 penalty term;- `'elasticnet'`: both L1 and L2 penalty terms are added... warning:: Some penalties may not work with some solvers. See the parameter `solver` below, to know the compatibility between the penalty and solver... versionadded:: 0.19 l1 penalty with SAGA solver (allowing 'multinomial' + L1).. deprecated:: 1.8 `penalty` was deprecated in version 1.8 and will be removed in 1.10. Use `l1_ratio` instead. `l1_ratio=0` for `penalty='l2'`, `l1_ratio=1` for `penalty='l1'` and `l1_ratio` set to any float between 0 and 1 for `'penalty='elasticnet'`.",'deprecated'
,"C C: float, default=1.0Inverse of regularization strength; must be a positive float.Like in support vector machines, smaller values specify strongerregularization. `C=np.inf` results in unpenalized logistic regression.For a visual example on the effect of tuning the `C` parameterwith an L1 penalty, see::ref:`sphx_glr_auto_examples_linear_model_plot_logistic_path.py`.",1.0
,"l1_ratio l1_ratio: float, default=0.0The Elastic-Net mixing parameter, with `0 <= l1_ratio <= 1`. Setting`l1_ratio=1` gives a pure L1-penalty, setting `l1_ratio=0` a pure L2-penalty.Any value between 0 and 1 gives an Elastic-Net penalty of the form`l1_ratio * L1 + (1 - l1_ratio) * L2`... warning:: Certain values of `l1_ratio`, i.e. some penalties, may not work with some solvers. See the parameter `solver` below, to know the compatibility between the penalty and solver... versionchanged:: 1.8 Default value changed from None to 0.0... deprecated:: 1.8 `None` is deprecated and will be removed in version 1.10. Always use `l1_ratio` to specify the penalty type.",0.0
,"dual dual: bool, default=FalseDual (constrained) or primal (regularized, see also:ref:`this equation `) formulation. Dual formulationis only implemented for l2 penalty with liblinear solver. Prefer `dual=False`when n_samples > n_features.",False
,"tol tol: float, default=1e-4Tolerance for stopping criteria.",0.0001
,"fit_intercept fit_intercept: bool, default=TrueSpecifies if a constant (a.k.a. bias or intercept) should beadded to the decision function.",True
,"intercept_scaling intercept_scaling: float, default=1Useful only when the solver `liblinear` is usedand `self.fit_intercept` is set to `True`. In this case, `x` becomes`[x, self.intercept_scaling]`,i.e. a ""synthetic"" feature with constant value equal to`intercept_scaling` is appended to the instance vector.The intercept becomes``intercept_scaling * synthetic_feature_weight``... note:: The synthetic feature weight is subject to L1 or L2 regularization as all other features. To lessen the effect of regularization on synthetic feature weight (and therefore on the intercept) `intercept_scaling` has to be increased.",1
,"class_weight class_weight: dict or 'balanced', default=NoneWeights associated with classes in the form ``{class_label: weight}``.If not given, all classes are supposed to have weight one.The ""balanced"" mode uses the values of y to automatically adjustweights inversely proportional to class frequencies in the input dataas ``n_samples / (n_classes * np.bincount(y))``.Note that these weights will be multiplied with sample_weight (passedthrough the fit method) if sample_weight is specified... versionadded:: 0.17 *class_weight='balanced'*",None
,"random_state random_state: int, RandomState instance, default=NoneUsed when ``solver`` == 'sag', 'saga' or 'liblinear' to shuffle thedata. See :term:`Glossary ` for details.",None
,"solver solver: {'lbfgs', 'liblinear', 'newton-cg', 'newton-cholesky', 'sag', 'saga'}, default='lbfgs'Algorithm to use in the optimization problem. Default is 'lbfgs'.To choose a solver, you might want to consider the following aspects:- 'lbfgs' is a good default solver because it works reasonably well for a wide class of problems.- For :term:`mul

In [13]:
weights = pd.DataFrame(
    model.coef_,
    columns=X.columns,
    index=model.classes_
)

weights


,char_addition,char_omission,char_substitution,has_addition,has_omission,has_substitution,word_count_diff,has_spacing_issue,has_diacritic_loss
Orthographic Instability,-0.030165,0.041178,-0.148560,-0.873426,0.399034,-0.324858,-0.476790,0.396214,0.146793
Phonetic Confusion,0.051930,-0.131665,0.002082,-0.043548,-0.432268,0.421180,-0.117098,-0.093267,0.083156
Word Boundary Confusion,-0.021765,0.090487,0.146478,0.916974,0.033234,-0.096323,0.593888,-0.302947,-0.229949


In [14]:
from sklearn.metrics import classification_report

y_pred = model.predict(X_test)
print(classification_report(y_test, y_pred))


                          precision    recall  f1-score   support

Orthographic Instability       0.70      0.73      0.71      1570
      Phonetic Confusion       0.51      0.63      0.56       926
 Word Boundary Confusion       0.35      0.07      0.11       342

                accuracy                           0.61      2838
               macro avg       0.52      0.47      0.46      2838
            weighted avg       0.59      0.61      0.59      2838



In [15]:
# Convert learned weights to a nested dict
pattern_feature_weights = {
    pattern: weights.loc[pattern].to_dict()
    for pattern in weights.index
}

pattern_feature_weights


{'Orthographic Instability': {'char_addition': -0.030164820670746512,
  'char_omission': 0.04117811041825914,
  'char_substitution': -0.1485601851661617,
  'has_addition': -0.8734261252938215,
  'has_omission': 0.39903437898767374,
  'has_substitution': -0.32485751149323555,
  'word_count_diff': -0.4767899616481992,
  'has_spacing_issue': 0.3962141171840923,
  'has_diacritic_loss': 0.1467928601760642},
 'Phonetic Confusion': {'char_addition': 0.0519300435189989,
  'char_omission': -0.1316652033902746,
  'char_substitution': 0.0020821221107781107,
  'has_addition': -0.04354826804585802,
  'has_omission': -0.4322680700687417,
  'has_substitution': 0.42118031166199627,
  'word_count_diff': -0.11709827357067043,
  'has_spacing_issue': -0.09326698929698049,
  'has_diacritic_loss': 0.08315639947287486},
 'Word Boundary Confusion': {'char_addition': -0.021765222848248042,
  'char_omission': 0.09048709297199914,
  'char_substitution': 0.14647806305544991,
  'has_addition': 0.9169743933397061,


In [16]:
import json
from pathlib import Path

ARTIFACT_DIR = Path("../artifacts")
ARTIFACT_DIR.mkdir(exist_ok=True)

weights_path = ARTIFACT_DIR / "ml_learned_feature_weights.json"

with open(weights_path, "w", encoding="utf-8") as f:
    json.dump(pattern_feature_weights, f, indent=2, ensure_ascii=False)

print(f"Saved ML feature weights to: {weights_path.resolve()}")


Saved ML feature weights to: D:\RP\sinhala-dyslexic-writing-pattern-classifier\writing_pattern_classifier\artifacts\ml_learned_feature_weights.json


In [17]:
print("Notebook frozen. Artifacts saved:")
print("- artifacts/ml_learned_feature_weights.json")
print("- artifacts/ml_weight_metadata.json")


Notebook frozen. Artifacts saved:
- artifacts/ml_learned_feature_weights.json
- artifacts/ml_weight_metadata.json
